# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [27]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [28]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [29]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [30]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7889

To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [34]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [35]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [43]:
get_ticket_price_tool = {
    "type": "function",
    "function": {
        "name": "get_ticket_price",
        "description": "Returns the ticket price for the given destination city.",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "Name of the destination city"
                }
            },
            "required": ["destination_city"]
        }
    }
}


## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [37]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [38]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [39]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7890

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Paris
Tool get_ticket_price called for Tokyo
Tool get_ticket_price called for Berlin
Tool get_ticket_price called for İstanbul


burdan sonraki ksıım ucak rezervasyonu için ekeldim

In [47]:
make_reservation_tool = {
    "type": "function",
    "function": {
        "name": "make_reservation",
        "description": "Makes a flight reservation for a given destination city.",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "Name of the destination city to reserve a flight for"
                }
            },
            "required": ["destination_city"]
        }
    }
}


In [48]:
tools = [get_ticket_price_tool, make_reservation_tool]


In [49]:
def make_reservation(destination_city):
    print(f"Reservation made for {destination_city}")
    return f"Your flight to {destination_city.title()} on July 14 has been successfully reserved."


In [50]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    tool_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    if tool_name == "get_ticket_price":
        city = arguments.get("destination_city")
        price = get_ticket_price(city)
        content = json.dumps({"destination_city": city, "price": price})

    elif tool_name == "make_reservation":
        city = arguments.get("destination_city")
        result = make_reservation(city)
        content = json.dumps({"message": result})

    else:
        content = json.dumps({"message": "Unknown tool"})

    response = {
        "role": "tool",
        "content": content,
        "tool_call_id": tool_call.id
    }

    return response, arguments.get("destination_city")


In [51]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7892

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Tokyo
Reservation made for Tokyo
